## Convert GDB to PostGIS

In [19]:
%%bash
campus_gdb=~/work/test-data/campus.gdb
ogrinfo $campus_gdb

Had to open data source read-only.
INFO: Open of `/Users/yonghah/work/test-data/campus.gdb'
      using driver `OpenFileGDB' successful.
1: Reference (3D Multi Line String)
2: Room (3D Multi Polygon)
3: Floor (3D Multi Polygon)
4: Building (3D Multi Polygon)
5: DEPTS (None)
6: MATCH_ROOM_DEPT (None)
7: MATCH_ROOM_OCCUPANT (None)


In [24]:
# need to check this gives good curves
%%bash 
campus_gdb=~/work/test-data/campus.gdb
ogr2ogr -f "PostgreSQL" PG:"dbname=yonghah user=yonghah password=1111" -nln building -overwrite $campus_gdb Building
ogr2ogr -f "PostgreSQL" PG:"dbname=yonghah user=yonghah password=1111" -nln floor -overwrite $campus_gdb Floor
ogr2ogr -f "PostgreSQL" PG:"dbname=yonghah user=yonghah password=1111" -nln room -overwrite $campus_gdb Room
ogr2ogr -f "PostgreSQL" PG:"dbname=yonghah user=yonghah password=1111" -nln reference -overwrite $campus_gdb Reference

In [3]:
import psycopg2
import pandas as pd
conn = psycopg2.connect("dbname='yonghah' user='yonghah' host='localhost' password='1111'")

In [30]:
cur = conn.cursor()
cur.execute("""SELECT distinct rmtyp, rmtypdescr from room order by rmtyp""")

In [31]:
df = pd.DataFrame(cur.fetchall(), columns=['rmtyp', 'rmtypdescr'])
df

,rmtyp,rmtypdescr
0,010,Custodial
1,020,Circulat'n
2,030,Mechanical
3,040,PubRestRm
4,050,Inactive
5,060,Renovation
6,070,Unfinished
7,110,Classroom
8,115,Class Serv
9,210,Class Lab


In [34]:
conn = psycopg2.connect("dbname='yonghah' user='yonghah' host='localhost' password='1111'")
cur = conn.cursor()
cur.execute("""SELECT count(*) from room where rmtyp='310'""")
cur.fetchall()

[(15335,)]

## Let's pick up non-restroom rooms 
- not restroom 040, 919
- not circulation (020 or (315 and sub 08)) 
- around 1700 rooms

In [35]:
conn = psycopg2.connect("dbname='yonghah' user='yonghah' host='localhost' password='1111'")
cur = conn.cursor()
cur.execute("""select count(rmrecnbr) from (
    SELECT rmrecnbr from room tablesample system(4)\
    where rmtyp != '020' and not (rmtyp ='315' and rmsubtyp = '08') \
    and not (rmtyp in ('040', '919')) LIMIT 2000) \
    as foo""")
cur.fetchall()

[(1871,)]

## Separate validation dataset

In [22]:
import os
import shutil
import random

In [62]:
rmtyp = 'other'
all_path = "../image/rmtyp/" + rmtyp
train_dst = "../image/data/train/" + rmtyp
val_dst = "../image/data/validate/" + rmtyp
ftype = "png"
random_fraction = 0.3

In [63]:
files = [file for file in os.listdir(all_path) if os.path.isfile(os.path.join(all_path, file)) and file.split('.')[-1] == ftype]
len(files)

1856

In [64]:
random_amount = round(len(files) * random_fraction)
random.shuffle(files)
random_amount

557

In [65]:
# check dst directory
dsts = [train_dst, val_dst]
for dst in dsts:
    if os.path.exists(dst):
        shutil.rmtree(dst)
    os.makedirs(dst)
    print(dst)

../image/data/train/other
../image/data/validate/other


In [66]:
for i, f in enumerate(files):
    dst = train_dst
    if i < random_amount:
        dst = val_dst
    shutil.copy(os.path.join(all_path, f), dst)